This code implements offensive language detection in Hebrew text using the methodology from the research paper
https://aclanthology.org/2022.lrec-1.396/,
combined with specific examples for training. It uses a multi-feature approach that combines:

BERT embeddings (using multilingual BERT) for understanding sentence context
Character n-grams for capturing word patterns
Context analysis specifically designed for Hebrew words that can be either offensive or non-offensive depending on context (like "נוכל" which can mean either "we can" or "scoundrel")

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from tqdm import tqdm
import os


In [ ]:
example_df = """
דוגמאות:

המשפט: אם בא בעל מניות וגנב ממפעל 25 מיליון שקלים. האם המדינה צריכה לתת את התשובה על זה? אני נותן דוגמה אבסטרקטית..
תשובה: לא, המילה גנב לא נאמרת באופן פוגעני


המשפט: "כלב שמירה זה מונח בסדר... כאן אנחנו מפגרים אחרי למעלה מ-40 מדינות..."
תשובה: לא


המשפט: אתה גנב וכולכם פה גונבים את המידה
תשובה: כן, המילה גנב נאמרה באופן פוגעני

המשפט:אולי נוכל לבשר את הדבר הזה. נגב גליל רוצה לתת ואי אפשר שביורוקרטיה כספית תעצור את זה, אז תבדוק למה זה לא קורה. אני אתן לך בהמשך להתייחס להחלטת הממשלה כי אני רוצה להמשיך עם משרדי ממשלה נוספים ואני אתן לך גם להשיב לנו על הנושא הזה.
תשובה: לא

המשפט: צריך לסתום את החומה כדי שלא יכנסו
תשובה: לא, המילה תסתום לא נאמרה בפירוש פוגעני


המשפט: תסתום את הפה עכשיו. חצוף שכמוך. אתה בא לאיים עליי כל הזמן? אתה חושב שאני מפחד ממך?
תשובה: כן, המילה תסתום נאמרה בפירוש פוגעני


המשפט: אנחנו נוכל לבנות פה הרבה דברים אך לא הכל
תשובה: לא, המילה נוכל לא נאמרה בפירוש בפוגעני אלא כפועל


המשפט: נוכל לבקר אותך מחר אם מתאים לך
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: נוכל להגיע בזמן אם נצא עכשיו
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: אני חושב שנוכל למצוא פתרון יחד
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: נוכל לסיים את הפרויקט השבוע
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: האם נוכל להיפגש מחר בבוקר?
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת

המשפט:תקשיבו, אתם יודעים עד כמה זה הזוי? הדוברת הבאה שהתכוונתי לתת לה את רשות הדיבור, כמו שעשינו בדיונים האחרונים, זו הגב' ורוניקה.
תשובה: כן

המשפט: הוא נוכל ורמאי שלא ניתן לסמוך עליו
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: תיזהר ממנו, הוא נוכל מהיום שנולד
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: אל תאמין למילה שלו, סתם נוכל
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: איזה נוכל אתה, תתבייש
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: הנוכל הזה רימה את כולם
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: נוכל לפתח מיומנויות חדשות
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: נוכל להעצים את היכולות שלנו
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת


המשפט: אתה נוכל וגנב, לך מפה
תשובה: כן, המילה נוכל משמשת ככינוי גנאי


המשפט: הוא נוכל מושחת לחלוטין
תשובה: כן, המילה נוכל משמשת ככינוי גנאי

משפט: לא. רק רגע. אנחנו נוכל לדייק את הנתונים.
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת

משפט: אבל יש לך נגיד יותר פחי זבל וכאלה? אתה מתייחס לזה?
תשובה: לא

משפט: איזה שמונה דגמים? איזה רוחב הם? למה אתה זורה להם חול בעיניים? למה כל הזמן לשקר? למה אתם משקרים? אתם לא מפסיקים לשקר.
תשובה: כן

משפט: תתבע אותי, תקלל אותי כמו תמיד, חבורת נוכלים
תשובה: כן

משפט: עד היום יש אנשים מוכשרים שמגיעים לארץ ולא יכולים לעבוד במקצוע שלהם, וזה דבר הזוי.
תשובה: לא

משפט:אני לא יודע לומר אפילו מבחינה תכנונית איך זה עובד – של מה נשלח לדואר זבל ומה לא.
תשובה: לא


משפט:ישראלים יכולים להגיע לארצות הברית מכוח נישואים, ולקבל שם מעמד. עכשיו אנחנו אומרים לאזרחים אמריקנים שהם לא יכולים לקבל מעמד בישראל. זה הזוי.
תשובה: כן

אז תבדוק לי. אני מבקש ספטמבר, תן לי אוגוסט. רגע, לא להפריע לי. אוחיון, תבדוק בחוץ, תעשה טלפונים ואולי נוכל לבשר את זה לפחות.
תשובה: לא


משפט:אתה יודע מה, אצלכם הכול הולך לזווית מגעילה כזאת.
תשובה: כן


משפט:אתה יודע מה, אצלכם הכול הולך לזווית מגעילה כזאת.
תשובה: כן

משפט:אבל אין סיבה לא להזניק אותם, וברגע שניתן להזניק אותם, גברתי, אז הם גם יכולים לשבת במוקד? זה הזוי.
תשובה: כן


משפט:ותסלחו לי - אני לא מרגיש טוב מכל השאלות שלכם. אני לא משקר.
תשובה: לא


משפט:הדבר השני, המשרד להגנת הסביבה אמר מלכתחילה, הסתייג מהתכנית, כי אין פתרון קצה ולקראת חידוש התוקף נאמרו דברים דומים - - - לכמויות אדירות של זבל, מדובר כנראה במאות אלפי עופות.
תשובה: לא

משפט:נושא פינוי הזבל. חוות לולים שהיא שייכת לרפורמה, בעצם חוות לולים מודרנית כזו, יש בה מסועים לפינוי של הזבל.
תשובה: לא

משפט:אז למה בונים אותה? אם אין פתרון קצה לפינוי זבל, אז למה בונים אותה?
תשובה: לא

משפט:גם להגיש 48 אלף הסתייגויות זה מסריח.
תשובה: לא

משפט:אתם פחדנים. פחדנים ובריונים.
תשובה: כן

משפט:גם אם זה חוקי, זה מסריח ואתה יודע את זה.
תשובה: כן

משפט:חבר הכנסת קיש, אתה נקרא לסדר פעם שלישית. קראת ליושב ראש הישיבה שקרן.
תשובה: כן

משפט:זה מספיק לי, חאלס.
תשובה: כן

משפט:המועצה תקבל את הדיווחים על האוהלים, המפיק, לפעמים משתנות לו הדרישות, לפעמים משתנים לו דברים. לכן זה לא שמישהו יושב רגל על רגל, מעשן איזה סיגר מסריח, כמו שמישהו נוהג לעשות ומחליט זה, חס וחלילה.
תשובה: כן

משפט:אז אני אומר שאני מקווה שעד סוף החודש נוכל להגיע להבנה עם משרד המשפטים ולהפיץ את התזכיר.
תשובה: לא

משפט:מבחינתי אנחנו נוכל לדווח לכם כשיסתיים הבירור שלנו. כאמור, מבחינתנו, מבחינה מהותית, העבודה על החוק הסתיימה, כולל התייחסות להערות של משרד המשפטים שכבר קיבלנו.
תשובה: לא

משפט:נוכל לשמוע אתכם.
תשובה: לא

משפט:רגע, רגע, תהיי בריאה, את מפחידה אותם. אני לא רוצה לגמור להם את כל ה-19 מיליארד. אחר כך נגמור אותם. אני מדבר רק על נהריה. איך את רצה לשם?
תשובה: לא

משפט:אז בגלל שיש פה אנשים זבלים שרוצים לנצל נשים אוקראיניות צעירות, אז היא לא תוכל לבוא לפה ולהיות במקום בטוח?
תשובה: כן

משפט:בעיקר שקרן ומסית ומחולל שנאה.
תשובה: כן

משפט:לא, בואו תאפשרו לנו יחד, תאגדו איתנו את הכוחות כדי למנוע את העוולה הזו. אתם הולכים להנציח בעיה שקיימת במדינת ישראל בצורה מזוויעה על כל הספקטרום שלה.
תשובה: לא

משפט:אתה עשית תרגיל מסריח.
תשובה: כן

משפט:די, מספיק, בשבילכם כל העצמאים רמאים. די, מספיק, אתם מתחילים להגזים עם זה. מספיק, עבורכם כל העצמאים רמאים? די, מספיק, בושה וחרפה.
תשובה: כן

משפט:אנחנו כוועדה לא נוכל לתת יד למשהו שיתקדם ובחזית של העורף - הולכים אחורה. לא יהיה, לא בבית ספרי, לא ניתן לתלמידים להיות כמו שראינו למעלה.
תשובה: לא

המשפט: נוכל לבנות כאן משהו נפלא
תשובה: לא, המילה נוכל משמשת כפועל המציין יכולת

המשפט: "אל תאמין למילה שלו, הוא נוכל מהיום שנולד"
תשובה: כן, המילה נוכל משמשת כמילת גנאי לתיאור אדם לא אמין

המשפט: "נוכל לבקר אותך מחר אם מתאים לך"
תשובה: לא, המילה נוכל מציינת אפשרות או יכולת לעשות פעולה

המשפט:"גופים אלה ייבחרו על-ידי הנוער במהלך החודש הקרוב, ביניהם נוכל למצוא את התיכון, תנועת הנוער או היחידה הצבאית"
תשובה: לא, המילה נוכל מציינת אפשרות או יכולת לעשות פעולה

המשפט: "הוא נוכל ידוע לשמצה"
תשובה: כן, המילה נוכל משמשת כמילת גנאי

המשפט: "נוכל לשפר את התהליכים"
תשובה: לא, המילה נוכל מציינת אפשרות או יכולת

המשפט: "איזה נוכל חסר מצפון"
תשובה: כן, המילה נוכל משמשת כמילת גנאי

המשפט: "נוכל להגיע להישגים משמעותיים"
תשובה: לא, המילה נוכל מציינת אפשרות או יכולת

המשפט: "תיזהרו מהנוכל הזה"
תשובה: כן, המילה נוכל משמשת כמילת גנאי

המשפט: "נוכל לקדם את הפרויקט"
תשובה: לא, המילה נוכל מציינת אפשרות או יכולת

המשפט: "נוכל מרושע שכמוך"
תשובה: כן, המילה נוכל משמשת כמילת גנאי
"""

In [ ]:
def parse_examples(text):
    """Parse examples from the provided format"""
    examples = []
    current_text = ""
    current_label = None

    lines = text.strip().split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('המשפט:') :
            current_text = line.replace('המשפט:', '').strip().strip('"')
        elif line.startswith('משפט:') :
            current_text = line.replace('משפט:', '').strip().strip('"')
        elif line.startswith('תשובה:'):
            answer = line.replace('תשובה:', '').strip()
            # If answer contains 'כן', it's offensive
            current_label = 1 if 'כן' in answer else 0
            if current_text:
                examples.append({
                    'text': current_text,
                    'label': current_label,
                })
                current_text = ""
                current_label = None

    return examples

In [ ]:
class ContextAnalyzer:
    """Enhanced context analyzer based on detailed Hebrew rules"""
    def __init__(self):
        self.dual_meaning_words = {
            'נוכל': {
                'verb_indicators': ['ללכת', 'לבוא', 'לבנות', 'לעשות', 'לבקר', 'למצוא', 'אנחנו', 'הם'],
                'noun_indicators': ['הוא', 'היא', 'שכמוך', 'מרושע', 'ורמאי','יא']
            },
            'מפגרים': {
                'verb_indicators': ['אחרי', 'בזמן', 'בלוח'],
                'noun_indicators': ['אתם', 'הם', 'אתה']
            },
        }

        self.warning_context = [
            'תיזהר ממנו',
            'אל תאמין',
            'תיזהר מ',
            'אל תתקרב'
        ]

        self.negative_descriptors = [
            'מרושע',
            'ורמאי',
            'שכמוך',
            'חצוף'
        ]

        self.positive_context = {
            'planning_words': ['נוכל ל', 'אפשר ל', 'ניתן ל'],
            'ability_words': ['יכולת', 'אפשרות', 'יכולים'],
            'future_actions': ['בקרוב', 'מחר', 'בהמשך']
        }

    def get_features(self, text):
        """Get all context features for a given text"""
        features = {}

        # Check dual meaning words
        for word in self.dual_meaning_words:
            if word in text:
                is_verb = any(indicator in text for indicator in self.dual_meaning_words[word]['verb_indicators'])
                is_noun = any(indicator in text for indicator in self.dual_meaning_words[word]['noun_indicators'])
                features[f'{word}_is_verb'] = float(is_verb)
                features[f'{word}_is_noun'] = float(is_noun)

        # Warning features
        features['has_warning'] = float(any(warning in text for warning in self.warning_context))
        features['warning_count'] = float(sum(warning in text for warning in self.warning_context))

        # Descriptor features
        features['has_negative_desc'] = float(any(desc in text for desc in self.negative_descriptors))
        features['negative_desc_count'] = float(sum(desc in text for desc in self.negative_descriptors))

        # Positive context features
        features['has_planning'] = float(any(word in text for word in self.positive_context['planning_words']))
        features['has_ability'] = float(any(word in text for word in self.positive_context['ability_words']))
        features['has_future'] = float(any(word in text for word in self.positive_context['future_actions']))

        # Additional specific rules
        features.update({
            'starts_with_verb': float(any(text.startswith(word) for word in ['נוכל', 'אפשר', 'ניתן'])),
            'has_first_person_plural': float('אנחנו' in text or any(word.startswith('נוכל ל') for word in text.split())),
            'describes_person': float(any(marker + ' נוכל' in text for marker in ['הוא', 'היא', 'אתה', 'אתם'])),
            'followed_by_action': float(any(f'נוכל {word}' in text for word in ['ל', 'את', 'עם']))
        })

        # Ensure all features are float type
        features = {k: float(v) for k, v in features.items()}

        return features

In [ ]:
class HebrewOffensiveDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, char_vectorizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.char_vectorizer = char_vectorizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        # BERT tokenization
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        # Character n-grams
        char_features = self.char_vectorizer.transform([text]).toarray()

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'char_ngrams': torch.tensor(char_features[0], dtype=torch.long),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
class MultiFeatureModel(nn.Module):
    def __init__(self, bert_hidden_size=768, char_vocab_size=1000, n_gram_dim=100):
        super().__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.char_embedding = nn.Embedding(char_vocab_size, n_gram_dim)

        combined_dim = bert_hidden_size + n_gram_dim
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 2)
        )

    def forward(self, input_ids, attention_mask, char_ngrams):
        # BERT features
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask)
        bert_features = bert_outputs.pooler_output

        # Character n-gram features
        char_features = self.char_embedding(char_ngrams)
        char_features = torch.mean(char_features, dim=1)

        # Combine features
        combined = torch.cat([bert_features, char_features], dim=1)
        return self.classifier(combined)


In [ ]:
def train_model(train_dataloader, model, device, num_epochs=10):
    """Train the model"""
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    criterion = nn.CrossEntropyLoss()

    print(f"\nStarting training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}')
        for batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            char_ngrams = batch['char_ngrams'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask, char_ngrams)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * correct / total:.2f}%'
            })

        avg_loss = total_loss / len(train_dataloader)
        accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [ ]:
def process_batch(model, batch_num, base_path, tokenizer, char_vectorizer, device):
    """Process only sentences that contain potential offensive words"""
    try:
        # Read batch file
        file_path = f"{base_path}/batch_{batch_num}_with_offensive_words.csv"
        df = pd.read_csv(file_path)

        # Clean NaN values and filter relevant rows
        print(f"\nBatch {batch_num} - Shape before cleaning: {df.shape}")
        df['conversation'] = df['conversation'].fillna('')
        df = df[df['conversation'] != '']

        # Only process rows where contain_offensive_words == 1
        df = df[df['contain_offensive_words'] == 1]
        print(f"Batch {batch_num} - Sentences to classify: {len(df)}")

        if len(df) == 0:
            print(f"No sentences to classify in batch {batch_num}")
            return

        # Create dataset for classification
        dataset = HebrewOffensiveDataset(
            df['conversation'].tolist(),
            [0] * len(df),  # Dummy labels for prediction
            tokenizer,
            char_vectorizer
        )
        dataloader = DataLoader(dataset, batch_size=16)

        # Get predictions
        model.eval()
        all_predictions = []
        all_probabilities = []

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                char_ngrams = batch['char_ngrams'].to(device)

                outputs = model(input_ids, attention_mask, char_ngrams)
                probabilities = torch.softmax(outputs, dim=1)
                predictions = torch.argmax(outputs, dim=1)

                all_predictions.extend(predictions.cpu().numpy())
                all_probabilities.extend(probabilities.cpu().numpy())

        # Add predictions to dataframe
        df['is_offensive'] = all_predictions
        df['confidence'] = [prob[1] for prob in all_probabilities]

        # Create output directories if they don't exist
        os.makedirs("predictions/offensive", exist_ok=True)
        os.makedirs("predictions/non_offensive", exist_ok=True)

        # Split and save results
        offensive_df = df[df['is_offensive'] == 1]
        non_offensive_df = df[df['is_offensive'] == 0]

        if len(offensive_df) > 0:
            offensive_df.to_csv(f"predictions/offensive/batch_{batch_num}_offensive.csv", index=False)
        if len(non_offensive_df) > 0:
            non_offensive_df.to_csv(f"predictions/non_offensive/batch_{batch_num}_non_offensive.csv", index=False)

        print(f"Results for batch {batch_num}:")
        print(f"- Found {len(offensive_df)} offensive conversations")
        print(f"- Found {len(non_offensive_df)} non-offensive conversations")

    except Exception as e:
        print(f"Error processing batch {batch_num}: {str(e)}")

In [ ]:
def main():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Parse training examples
    print("Parsing training examples...")
    examples = parse_examples(example_df)

    # Extract texts and labels
    train_texts = [ex['text'] for ex in examples]
    train_labels = [ex['label'] for ex in examples]

    # Print training data summary
    print(f"\nTraining Data Summary:")
    print(f"Total examples: {len(train_texts)}")
    print(f"Offensive examples: {sum(train_labels)}")
    print(f"Non-offensive examples: {len(train_labels) - sum(train_labels)}")

    # Initialize components
    print("\nInitializing model components...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Create character n-gram vectorizer
    print("Creating character n-gram vectorizer...")
    char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 5), max_features=1000)
    char_vectorizer.fit(train_texts)

    # Create training dataset
    train_dataset = HebrewOffensiveDataset(
        train_texts,
        train_labels,
        tokenizer,
        char_vectorizer
    )
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    # Initialize model
    print("Initializing model...")
    model = MultiFeatureModel().to(device)

    # Train model
    train_model(train_dataloader, model, device)

    # Process all batches
    print("\nProcessing batches...")
    base_path = "/home/gorelikk/NLP-PROJECT/Pre_Process/Data Heuristics/Data/Harmful"

    for batch_num in range(6, 10):
        print(f"\nProcessing batch {batch_num}")
        process_batch(
            model,
            batch_num,
            base_path,
            tokenizer,
            char_vectorizer,
            device
        )

    # Save model
    print("\nSaving model...")
    torch.save({
        'model_state_dict': model.state_dict(),
        'char_vectorizer': char_vectorizer,
    }, 'hebrew_offensive_model.pt')
    print("Model saved successfully")


In [ ]:
if __name__ == "__main__":
    main()

Using device: cuda
Parsing training examples...

Training Data Summary:
Total examples: 63
Offensive examples: 30
Non-offensive examples: 33

Initializing model components...
Creating character n-gram vectorizer...
Initializing model...

Starting training...


Epoch 1/10: 100%|██████████| 4/4 [00:00<00:00,  5.61it/s, loss=0.6805, acc=52.38%]


Epoch 1 - Average Loss: 0.6939, Accuracy: 52.38%


Epoch 2/10: 100%|██████████| 4/4 [00:00<00:00, 15.49it/s, loss=0.6868, acc=65.08%]


Epoch 2 - Average Loss: 0.6850, Accuracy: 65.08%


...

Epoch 10/10: 100%|██████████| 4/4 [00:00<00:00, 15.46it/s, loss=0.2176, acc=100.00%]


Epoch 10 - Average Loss: 0.2271, Accuracy: 100.00%

Processing batches...

Processing batch 6

Batch 6 - Shape before cleaning: (504450, 7)
Batch 6 - Sentences to classify: 2832


../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [859,0,0], thread: [124,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [859,0,0], thread: [125,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [859,0,0], thread: [126,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [859,0,0], thread: [127,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [939,0,0], thread: [84,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [939,0,0], thread: [85,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [

Error processing batch 6: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Processing batch 7

Batch 7 - Shape before cleaning: (284263, 7)
Batch 7 - Sentences to classify: 1342
Error processing batch 7: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Processing batch 8

Batch 8 - Shape before cleaning: (556595, 7)
Batch 8 - Sentences to classify: 3355
Error processing batch 8: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Add this before training to check available GPU memory
import torch
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

GPU Memory: 50.90 GB
